# Introduction

Let's consider an example of automatical genration of texts with using of Recurrent Neural Network (RNN).

* Useful meterials about RNN: [here](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
* PyTorch implementation: [here](https://github.com/karpathy/char-rnn).

In [1]:
import warnings
from typing import Iterable, Tuple

import pytorch_lightning as pl
import torch
from torch import nn

warnings.filterwarnings("ignore")

# Data preparation

## Anna Karenina text loading

In [2]:
with open("anna.txt", "r") as f:
    text = f.read()

text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

## Tokenization of the text

Let's make two dictionaries for transformation of symbols to integer numbers and reverse.

In [3]:
unique_chars = tuple(set(text))
int2char = dict(enumerate(unique_chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = torch.tensor([char2int[ch] for ch in text])
encoded[:100]

tensor([29, 10,  8, 56, 14, 38, 74,  4, 31, 37, 37, 37,  3,  8, 56, 56, 13,  4,
        15,  8, 35, 77, 11, 77, 38, 30,  4,  8, 74, 38,  4,  8, 11, 11,  4,  8,
        11, 77, 75, 38, 22,  4, 38, 58, 38, 74, 13,  4, 64, 44, 10,  8, 56, 56,
        13,  4, 15,  8, 35, 77, 11, 13,  4, 77, 30,  4, 64, 44, 10,  8, 56, 56,
        13,  4, 77, 44,  4, 77, 14, 30,  4, 26, 61, 44, 37, 61,  8, 13, 24, 37,
        37, 17, 58, 38, 74, 13, 14, 10, 77, 44])

Let's take a look on the scheme of char-RNN:
<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/charseq.jpeg?raw=1" width="30%">

The network waits **one-hot encoded** input. Below you can find the function to do it.

In [4]:
def one_hot_encode(int_words: torch.Tensor, n_labels: int) -> torch.Tensor:
    """
    Creates one-hot representation matrix for a given batch of integer sequences
    :param int_words: tensor of ints, which represents current sequence; shape: [batch_size, seq_len]
    :param n_labels: vocabulary size (number of unique tokens in data)
    :return: one-hot representation of the input tensor; shape: [batch_size, seq_len, n_labels]
    """
    words_one_hot = torch.zeros(
        (int_words.numel(), n_labels), dtype=torch.float32, device=int_words.device
    )
    words_one_hot[torch.arange(words_one_hot.shape[0]), int_words.flatten()] = 1.0
    words_one_hot = words_one_hot.reshape((*int_words.shape, n_labels))

    return words_one_hot

In [5]:
# testing the function
test_seq = torch.tensor([[3, 5, 1], [0, 2, 4]])
test_one_hot = one_hot_encode(test_seq, 8)

print(test_one_hot)

tensor([[[0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0.]],

        [[1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.]]])


## Batches making

As a simple example, batches looks like the following: we take encoded symbols and divide it into several sequences defined by a parameter `batch_size`. Each of our sequences has length `seq_length`.

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/sequence_batching@1x.png?raw=1" width=500px>

**1. Throwing out a piece of the text to make "whole" batches.**

Each batch contains $N \times M$ symbols where $N$ is number of sequences in batch (`batch_size`), $M$ — length of each sequence (`seq_length`). Then to obtain number of batches $K$ which we can make from the text, it's needed to divide length of the initial sequence by number of symbols in a batch. Then we can find the number of symbols which we should save in the text to form of $K$ batches: $N \times M \times K$.

**2. Dividing of the text into $N$ pieces**

This step is needed to have a possibility to going among the text by a window of size `[batch_size, seq_len]`. It can be implemented with simple `reshape`.

**3. When we have a text matrix we can move along it with the window to obtain batches**

From each position of the window we make training couples `(x, y)` the following way: $x$ - is all the elements of the window except the last column, $y$ - is all the lements of the window except the first column. Thereby for each token of initial text we will make prediction of the token following by this.  

In [6]:
def get_batches(
    int_words: torch.Tensor, batch_size: int, seq_length: int
) -> Iterable[torch.Tensor]:
    """
    Generates batches from encoded sequence.
    :param int_words: tensor of ints, which represents the text; shape: [batch_size, -1]
    :param batch_size: number of sequences per batch
    :param seq_length: number of encoded chars in a sequence
    :return: generator of pairs (x, y); x_shape, y_shape: [batch_size, seq_length - 1]
    """
    # 1. Truncate text, so there are only full batches
    window_size = seq_length + 1
    batch_size_total = batch_size * window_size
    n_batches = len(int_words) // batch_size_total
    int_words = int_words[: n_batches * batch_size_total]

    # 2. Reshape into batch_size rows
    int_words = int_words.reshape((batch_size, -1))

    # 3. Iterate through the text matrix
    for position in range(0, int_words.shape[1], window_size):
        x = int_words[:, position : position + window_size - 1]
        y = int_words[:, position + 1 : position + window_size]
        yield x, y

In [7]:
# testing the function
test_batches = get_batches(encoded, 8, 50)
test_x, test_y = next(test_batches)
assert test_x.shape == test_y.shape
print(f"x:\n{test_x[:10, :10]}\n")
print(f"y:\n{test_y[:10, :10]}")

x:
tensor([[29, 10,  8, 56, 14, 38, 74,  4, 31, 37],
        [56, 56, 38, 45,  4,  8, 44, 45,  4, 30],
        [14, 10,  8, 14,  4, 77, 44, 30, 26, 11],
        [38, 61,  4, 14, 10, 26, 64, 60, 10, 14],
        [38, 74, 73,  4, 10, 38,  4, 10,  8, 45],
        [74, 73,  4, 61, 10, 26,  4, 61,  8, 30],
        [30, 14,  4, 43, 38,  4, 76, 26, 58, 38],
        [73,  4, 23, 11, 38, 79, 38, 13,  4, 23]])

y:
tensor([[10,  8, 56, 14, 38, 74,  4, 31, 37, 37],
        [56, 38, 45,  4,  8, 44, 45,  4, 30, 10],
        [10,  8, 14,  4, 77, 44, 30, 26, 11, 64],
        [61,  4, 14, 10, 26, 64, 60, 10, 14, 15],
        [74, 73,  4, 10, 38,  4, 10,  8, 45,  4],
        [73,  4, 61, 10, 26,  4, 61,  8, 30,  4],
        [14,  4, 43, 38,  4, 76, 26, 58, 38, 74],
        [ 4, 23, 11, 38, 79, 38, 13,  4, 23, 11]])


## Class of the dataset

In [8]:
class AnnaData(torch.utils.data.IterableDataset):
    def __init__(self, int_words: torch.Tensor, batch_size: int, seq_length: int):
        self.int_words = int_words
        self.batch_size = batch_size
        self.seq_length = seq_length

    def __iter__(self):
        return get_batches(self.int_words, self.batch_size, self.seq_length)

# Model implementation

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/charRNN.png?raw=1" width=50%>

## Structure of the model

* Making and storing required dictionaries.
* Defining of layer [LSTM]((https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM)) with class `torch.nn.LSTM` instance, which takes parameters set: `input_size` — length of the sequence in a batch; `n_hidden` — size of the hidden layers; `n_layers` — number of the layers; `drop_prob` — dropout probability; и `batch_first` — a flag marking the fact that input sequences have batch dimension along the zero axis.
* Defining of Dropout layer with the same value `drop_prob`.
* Defining of fully connected layer with set of parameters: dimension of input — `n_hidden`; dimesion of output — dictionary size.
* And, finally, initializing of the weights and initial hidden state (`self.init_hidden()`).

In [9]:
class CharRNN(nn.Module):
    def __init__(
        self,
        unique_tokens: Tuple[str],
        n_hidden: int = 256,
        n_layers: int = 2,
        drop_prob: float = 0.5,
    ) -> None:
        super().__init__()
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.drop_prob = drop_prob

        # create mappings
        self.unique_tokens = unique_tokens
        self.int2char = dict(enumerate(self.unique_tokens))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}

        ## define the LSTM, dropout and fully connected layers
        self.lstm = nn.LSTM(
            len(self.unique_tokens),
            n_hidden,
            n_layers,
            dropout=drop_prob,
            batch_first=True,
        )
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(n_hidden, len(self.unique_tokens))

    def forward(
        self, x: torch.Tensor, hidden: torch.Tensor
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        r_output, hidden = self.lstm(x, hidden)
        out = self.dropout(r_output)
        # Stack up LSTM outputs using view. You may need to use contiguous to reshape the output.
        out = out.contiguous().view(-1, self.n_hidden)
        ## Get the output for classification.
        out = self.fc(out)
        return out, hidden

    def init_hidden(
        self, batch_size: int, weight_device: torch.device
    ) -> Tuple[torch.Tensor]:
        """
        Creates two new zero tensors for hidden state and cell state of LSTM
        :param batch_size: number of sequences per batch
        :param weight_device: torch.device("cuda") for GPU init or torch.device("cpu") for CPU init
        :return: tuple of two tensors of shape [n_layers x batch_size x n_hidden]
        """
        weight = next(self.parameters()).data
        hidden = (
            weight.new(self.n_layers, batch_size, self.n_hidden)
            .zero_()
            .to(weight_device),
            weight.new(self.n_layers, batch_size, self.n_hidden)
            .zero_()
            .to(weight_device),
        )
        return hidden

In [10]:
class CharRNNModule(pl.LightningModule):
    def __init__(
        self,
        unique_tokens: Tuple[str],
        n_hidden: int = 1024,
        n_layers: int = 2,
        drop_prob: float = 0.5,
        batch_size: int = 128,
        seq_length=256,
        lr: float = 0.001,
    ) -> None:
        super().__init__()
        self.model = CharRNN(unique_tokens, n_hidden, n_layers, drop_prob)
        self.hidden = None
        self.loss = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.n_chars = len(unique_tokens)
        self.batch_size = batch_size
        self.seq_length = seq_length

    def training_step(
        self, train_batch: Tuple[torch.Tensor, torch.Tensor]
    ) -> torch.Tensor:
        x, y = train_batch
        x, y = x.squeeze(0), y.squeeze(0)
        x = one_hot_encode(x, self.n_chars)

        if self.hidden is None:
            self.hidden = self.model.init_hidden(self.batch_size, self.device)
        self.hidden = tuple([each.data for each in self.hidden])

        output, self.hidden = self.model(x, self.hidden)
        loss = self.loss(output, y.reshape(self.batch_size * self.seq_length).long())

        self.log("train_loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return self.optimizer

## Model training

We'll use Adam optimizer and cross-entropy loss function. But we also have a coupe of peculiarities:
* While cycle we'll separate hidden state from its history because a hidden state of LSTM is a tuple of hidden conitions.
* We'll use gradient clipping to avoid "blowing" gradients.

In [11]:
# data
train_dataset = AnnaData(encoded, batch_size=128, seq_length=256)
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=1,  # batching is already implemented on our side
    shuffle=False,
    num_workers=1,  # don't change: it will lead to the wrong implementation
)
# model
char_rnn = CharRNNModule(unique_chars, n_hidden=1024, batch_size=128)
# trainer
trainer = pl.Trainer(max_epochs=15, gradient_clip_val=1.0, accelerator="gpu", devices=1)
trainer.fit(char_rnn, train_dataloaders=train_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name  | Type             | Params
-------------------------------------------
0 | model | CharRNN          | 13.0 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
13.0 M    Trainable params
0         Non-trainable params
13.0 M    Total params
52.097    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=15` reached.


# Model applying

Firstly, let's save the model to load it later. Below you can see the parameters required to making the same architecture, hyperparameters of the hidden layer and the tokens.

In [12]:
net = char_rnn.model
checkpoint = {
    "n_hidden": net.n_hidden,
    "n_layers": net.n_layers,
    "state_dict": net.state_dict(),
    "tokens": net.unique_tokens,
}

with open("rnn_x_epoch.net", "wb") as f:
    torch.save(checkpoint, f)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

## Making of predictions

To predict the continuation of the text, we transmit the last character to the network, it predicts the next character, which we again transmit to the input, receive another predicted character, and so on. Our predictions are based on a categorical probability distribution over all possible characters. We can limit the number of characters at each generation step to make the resulting predicted text more intelligent by considering only some of the most likely characters. On the one hand, this approach will allow us to consider not only the most probable sequence from the point of view of the model prediction. On the other hand, we will work with a limited set of generated options, so we will get rid of completely noisy predictions. You can find more [here](https://pytorch.org/docs/stable/generated/torch.topk.html#torch.topk).

In [13]:
def predict_next_char(
    model: torch.nn.Module, char: str, h: torch.Tensor = None, top_k: int = None
) -> Tuple[str, torch.Tensor]:
    """
    Given a character and a model, predicts next character in the sequence
    :param model: model that outputs next token probability distribution
    :param char: last character of the sequence to continue generation from
    :param h: hidden state of the model
    :param top_k: number of most probable tokens to be chosen from
    :return: tuple of next character and new hidden state
    """
    # tensor inputs
    x = torch.tensor([[model.char2int[char]]])
    x = one_hot_encode(x, len(model.unique_tokens))
    x = x.to(device)

    h = tuple([each.data for each in h])

    out, h = model(x, h)

    # get the character probabilities
    p = torch.nn.functional.softmax(out, dim=1).data.cpu()

    # get top characters
    if top_k is None:
        top_ch = torch.arange(len(model.unique_tokens))
    else:
        p, top_ch = p.topk(top_k)

    p.squeeze_()
    top_ch.squeeze_()
    char = top_ch[torch.multinomial(p / p.sum(), 1)]

    return model.int2char[char.item()], h

### Priming and text generation

We need to define a hidden state to avoid the network to generate random numbers.

In [14]:
def sample(model, size, prime="The", top_k=None):
    model.to(device)
    model.eval()

    # run through the prime characters
    chars = [ch for ch in prime]
    h = model.init_hidden(1, device)
    for ch in prime:
        char, h = predict_next_char(model, ch, h, top_k=top_k)

    chars.append(char)

    # pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict_next_char(model, chars[-1], h, top_k=top_k)
        chars.append(char)

    return "".join(chars)

In [15]:
print(sample(net, 1000, prime="Anna", top_k=5))

Anna that she crasped that whether series and herself thinking to him to a pondiat, that to say and a genting the best of harisa so mentilisy were
stonding that the was trun it asked with his wife to be said as the cream of a stees on the clang and troubbed the call the head. To such a child they was so such her side in his serious complexence in the secrath of
the sarrow, as
he had that, to be answering thin in which she sat delighting the party of the pain, where they
had been
talking in the
proppession who had telling one and so that it
was standing work to him, and seeing the marshal offorming, and the
perially thought of her landowned shills of the mothing, he went out and she said, would bury and horses of her
eyes. She said to his starming something so so in his father's and his brother's side.

"I am very been it's
so it was
to book?"

"The pity of all who did not bear. In the
long
weilent wine ther of the carreations, and tried to be delled, and the doctor and so a little more

## Checkpoint loading and generation

In [16]:
with open("rnn_x_epoch.net", "rb") as f:
    checkpoint = torch.load(f)

loaded = CharRNN(
    checkpoint["tokens"],
    n_hidden=checkpoint["n_hidden"],
    n_layers=checkpoint["n_layers"],
)
loaded.load_state_dict(checkpoint["state_dict"])

# sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said to her,
and and they could not have been thanking a little bangs. Seeding him with the light of those, so he could not have
said to this completening her
and, something and the convicion.

The state of the morting their carriage was shooking of the path, but his fellow had business who had standing in the clear, the
princess. "It's not a second with his wife, when
she
has now that it's so in answer, better in a tendroom,.'s
when he stopsed all the most signer and wisting to him, the complicht had always departure all the children. The passain he had
to do. This called to this for that seeming of the country and setting on a smold by that she changed the same again with a little back. "I don't walt to tee it, but it said to the past family. That's ther. I am not anything, but
I comes that he had not too went to this all that. It's a
long on the peasing, by a shall women with a meaning of," said Vronsky,
was nearer in
a confining with the morrow that she talked at
the cape